In [8]:
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup
import re
from selenium import webdriver
import time
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [26]:
driver = webdriver.Edge("C:\Program Files\msedgedriver.exe")
url = 'https://www.nfl.com/games/dolphins-at-bills-2023-reg-4?active-tab=watch'
options = Options()
options.headless = True

driver.get(url)
driver.maximize_window()
# drives_element = WebDriverWait(driver, 10).until(
#     EC.presence_of_all_elements_located((By.XPATH, r'\/\/\*\[@id="all-drives-panel"\]\/div\[1\]\/div\[[1-9]+\]\/div\/div\/dov\[2\]\/div\/div\/button\/div\/div\/div\[2\]\/div\/div\[2\]' ))
# )                                             

# print(drives_element)
# print(len(drives_element))

drives_element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "all-drives-panel"))
)

drives_html = drives_element.get_attribute('innerHTML')

# drive1 = driver.find_element(By.XPATH, '//*[@id="all-drives-panel"]/div[1]/div[2]/div/div/div[2]/div/div/button/div/div[2]/div/div[2]')
# drive1.click()
time.sleep(10)
driver.quit()




In [27]:
drives_html

'<div data-testid="gamecenter-drive-summary-list-container"><div class="css-a45wr7">1ST QUARTER</div><div><div class="css-7w6khc"><div class="css-view-175oi2r r-width-13qz1uu"><div class="css-view-175oi2r r-flexDirection-18u37iz r-width-13qz1uu"><div class="css-view-175oi2r r-alignItems-1awozwy r-height-3da1kt r-width-13qz1uu"><div class="css-view-175oi2r r-borderLeftColor-855088 r-borderRightColor-114ovsg r-borderStyle-1phboty r-height-1pi2tsx r-position-u8s1d" data-testid="trapezoid" style="border-bottom-color: rgb(0, 51, 141); border-bottom-width: 8px; width: 120px; border-left-width: 6px; right: 0px;"></div></div></div><div class="css-view-175oi2r r-width-13qz1uu r-boxShadow-1gmnr91 r-marginBottom-15zivkp r-borderRadius-z2wwpe" style="flex: 0 1 auto;"><div class="css-view-175oi2r r-backgroundColor-14lw9ot r-overflow-1udh08x r-borderRadius-z2wwpe r-width-13qz1uu" style="flex-direction: column;"><div class="css-view-175oi2r r-padding-t60dpp" style="flex: 0 1 auto;"><button role="butt